In [ ]:
# This notebook is to randomly select 500 points in ATL08 dataframe, 
# then buffer each point with 100 (bioclim) pixels (~100km) in x- and y-direction to form grdding box
# aggregate alt08 data to grid cells

In [1]:
import geopandas as gpd
import rioxarray as rxr

In [13]:
from shapely.geometry import box

In [ ]:
def get_bioclim_ref():
    biofn = "/adapt/nobackup/projects/ilab/data/worldclim/1km/bioclim/wc2.0_30s_bio/wc2.0_bio_30s_02.tif"
    ds = rxr.open_rasterio(biofn)
    csize = ds.rio.resolution()[0] # cell size of raster

    lons = ds.x.values # longitude array
    lats = ds.y.values # latitude array
return csize, lons, lats



In [ ]:
def fill_one_box(atl08_gdf, box):
    # box : footprint of grids

In [10]:


def create_cell(center_x, center_y, ref_raster, npix=100):
    # extract coords from reference raster
    ds = rxr.open_rasterio(ref_raster)
    lons = ds.x.values
    lats = ds.y.values
    xsize, ysize = ds.rio.resolution()
    
    
    # find nearest point in raster
    pick = ds.sel(x=center_x, y=center_y, method='nearest')
    xid = list(lons).index(pick.x.values)
    yid = list(lats).index(pick.y.values)
    
    # create x-axis and y-axis of cell
    xs = lons[(xid-npix):(xid+npix)]
    ys = lats[(yid-npix):(yid+npix)]
    
    return xs, ys

In [2]:
# read in atl08 gdf (dataset prepared by PM, with multiple filters)
fn = "/adapt/nobackup/people/jli30/workspace/forest/STG/data/atl08_2020_filtered.parquet"
c_gdf = gpd.read_parquet(fn)
print("Read Atl08 Completed")

Read Atl08 Completed


In [3]:
# randomly select points 
n_spl = 500
seed = 42
pts = c_gdf['geometry'].sample(n_spl, random_state=seed)

In [6]:
pts.iloc[0].x

-74.8553695678711

In [11]:
rst = "/adapt/nobackup/projects/ilab/data/worldclim/1km/bioclim/wc2.0_30s_bio/wc2.0_bio_30s_02.tif"
xx, yy = create_cell(pts.iloc[0].x, pts.iloc[0].y, rst, npix=10)

In [16]:
ds = rxr.open_rasterio(rst)
xsize, ysize= ds.rio.resolution()

In [12]:
xx, yy

(array([-74.9375    , -74.92916667, -74.92083333, -74.9125    ,
        -74.90416667, -74.89583333, -74.8875    , -74.87916667,
        -74.87083333, -74.8625    , -74.85416667, -74.84583333,
        -74.8375    , -74.82916667, -74.82083333, -74.8125    ,
        -74.80416667, -74.79583333, -74.7875    , -74.77916667]),
 array([55.8625    , 55.85416667, 55.84583333, 55.8375    , 55.82916667,
        55.82083333, 55.8125    , 55.80416667, 55.79583333, 55.7875    ,
        55.77916667, 55.77083333, 55.7625    , 55.75416667, 55.74583333,
        55.7375    , 55.72916667, 55.72083333, 55.7125    , 55.70416667]))

In [23]:
bx = list(map(lambda x, y: box(x, y, x+xsize, y+ysize), xx, yy))